1. Load the JSON data:

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/product_data.json", "dbfs:/FileStore/product_data.json")

True

In [0]:
df_json = spark.read.option("multiline", "true").json("/FileStore/product_data.json")
df_json.show(10)
df_json.printSchema()

+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------+-----+---------+-----------+-----+
|Electronics| 1200|      101|     Laptop|   35|
|Electronics|  800|      102| Smartphone|   80|
|  Furniture|  150|      103| Desk Chair|   60|
|Electronics|  300|      104|    Monitor|   45|
|  Furniture|  350|      105|       Desk|   25|
+-----------+-----+---------+-----------+-----+

root
 |-- Category: string (nullable = true)
 |-- Price: long (nullable = true)
 |-- ProductID: long (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Stock: long (nullable = true)



2. Data Cleaning:

In [0]:
# Filter by stock and category
df_filtered = df_json.filter(df_json['Stock'] >= 30).filter(df_json['Category'] == 'Electronics')
df_filtered.show()


+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------+-----+---------+-----------+-----+
|Electronics| 1200|      101|     Laptop|   35|
|Electronics|  800|      102| Smartphone|   80|
|Electronics|  300|      104|    Monitor|   45|
+-----------+-----+---------+-----------+-----+



3.Data Aggregation:

In [0]:
df_json.filter(df_json['Category'] == 'Furniture').agg({'Stock': 'sum'}).show()

df_avg_price = df_json.groupBy('Category').agg({'Price': 'avg'}).withColumnRenamed('avg(Price)', 'AvgPrice')
df_avg_price.show()

+----------+
|sum(Stock)|
+----------+
|        85|
+----------+

+-----------+-----------------+
|   Category|         AvgPrice|
+-----------+-----------------+
|Electronics|766.6666666666666|
|  Furniture|            250.0|
+-----------+-----------------+



4.Write the Data to JSON:

In [0]:
df_filtered.coalesce(1).write.json('/FileStore/cleaned_product_data.json')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1725955152151495>, line 1
----> 1 df_filtered.coalesce(1).write.json('/FileStore/cleaned_product_data.json')

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1927, in DataFrameWriter.json(self, path, mode, compression, dateFormat, timestampFormat, lineSep, encoding, ignoreNullFields)
   1918 self.mode(mode)
   1919 self._set_opts(
   1920     compression=compression,
   1921     dateFormat=dateFormat,
   (...)
   1925     ignoreNullFields=ignoreNull